# 組成ヒストグラム記述子の抽出

<br>
<div style="text-align: right;">
作成日2022年5月6日<br>                                                                     　作成者：小山翼 

引用：　R. Jalem, M. Nakayama, Y. Noda, T. Le, I. Takeuchi, Y. Tateyama, H. Yamasaki, "A general representation scheme for crystalline solids based on Voronoi-tessellation real feature values and atomic property data", Sci. Technol. Adv. Mater., 19, 231-242 (2018) <BR> [DOI: 10.1080/14686996.2018.1439253](https://doi.org/10.1080/14686996.2018.1439253)
<BR> GitHub: [NakayamaLab-NITech/composional-histogram-descriptor](https://github.com/NakayamaLab-NITech/composional-histogram-descriptor)

Example File (testdata.csv)引用元: R. Jalem et al. Chem. Mater., 24, 1357-1364 (2012) [DOI: 10.1021/cm3000427](https://doi.org/10.1021/cm3000427)



【要ユーザー操作】　ファイル名の定義、組成式が入っているラベルを指定する

In [12]:
inputfilename="testdata.csv"           #入力ファイル名
formula="formula"                      #組成式が入力されている列のラベル（１行目記載）を指定
outputfilename="out_comphist.csv"      #出力ファイル名

In [27]:
import re
import os
import numpy as np
import argparse
import math
import subprocess as sp
from subprocess import check_output
import pandas as pd
import shutil
import make_hist_in_ver3
import glob
from tqdm.notebook import tqdm

In [23]:
df = pd.read_csv(inputfilename)#ここでファイル名を変更する
print ("number of samples",len(df))
df.head()

number of samples 66


,Index#,formula,EA (eV),S/U
0,1,LiCaSbO4,NaN,NaN
1,2,LiMgPO4,0.30302,S
2,3,LiSrPO4,NaN,NaN
3,4,LiSrSbO4,NaN,NaN
4,5,LiCaAsO4,NaN,NaN


【以下　組成ヒストグラム記述子出力ルーチン】

In [28]:
%%time
path=os.getcwd()
d_list=[]
str_list=[]

for i in tqdm(range(len(df))):
    composition=df[formula][i]#ここで組成式が入っているラベルを指定

    parser = argparse.ArgumentParser(
    usage='python make_hist.py structure -his= -sigma= -kh='
        'composition         : enter a composition name (e.g. LiCoO2)'
        '-sigma=T          : sigma value for Gaussian broadning'
        '-kh=T             : output histgram descriptors considering two elements (Def. F)'
        'Histogram format  : 1:Prop (eg. AN, EN)  2: 3  3: prop_min  4: prop_max  5: bin_num  6: gaussian_sigma')

    parser.add_argument('composition',type=str)
    parser.add_argument('-sigma')
    parser.add_argument('-kh')


    args = parser.parse_args(args=[composition,'-kh=T'])#ここでオプションを設定する。
    reference = args.composition
    aparam2=args.sigma
    aparam3=args.kh
    reference = reference.split()
    if aparam2 is None:
        aparam2 = ''
    
    if aparam3 is None:
        aparam3 = ''
    df_columns=[]
    df_data=[]
    mk_hist=make_hist_in_ver3.compdescript(aparam2,aparam3,reference)
    comp_hist=["AN","EN","MP","PN","PG","MN","AW","AR","IR","CoR","CrR","spdf"]
    
    comp_list=[]
    comp_list1=[]
    comp_list2=[]
    for comp in comp_hist:
        comp_list.append(mk_hist[3][comp])
        comp_list1.append(mk_hist[4][comp])
        comp_list2.append(mk_hist[5][comp])
    
    df_last1=pd.concat(comp_list, axis=1)
    df_last2=pd.concat(comp_list1, axis=1)
    df_last3=pd.concat(comp_list2, axis=1)
            
    df_h = pd.concat([df_last1,df_last2,df_last3,mk_hist[0],mk_hist[1],mk_hist[2]], axis=1)
    d_list.append(df_h)
    str_list.append(reference[0])
#df2=pd.DataFrame(d_list,index=[i for i in range(len(df))])

df2=pd.concat(d_list,axis=0,ignore_index=True)
df2
#df_new=pd.merge(df,df2,left_index=True, right_index=True)
df_new=pd.concat([df,df2],axis=1)

df_new.to_csv(outputfilename,index=False)

df_new

  0%|          | 0/66 [00:00<?, ?it/s]

Wall time: 56.3 s


,Index#,formula,EA (eV),S/U,AN_0,AN_1,AN_2,AN_3,AN_4,AN_5,...,SPDFmatrix_0,SPDFmatrix_1,SPDFmatrix_2,SPDFmatrix_3,SPDFmatrix_4,SPDFmatrix_5,SPDFmatrix_6,SPDFmatrix_7,SPDFmatrix_8,SPDFmatrix_9
0,1,LiCaSbO4,NaN,NaN,0.0,0.0,0.142857,0.0,0.0,0.0,...,0.285714,2.000000,0.000000,0.000000,0.714286,0.000000,0.000000,0.000000,0.000000,0.000000
1,2,LiMgPO4,0.30302,S,0.0,0.0,0.142857,0.0,0.0,0.0,...,0.285714,2.000000,0.000000,0.000000,0.714286,0.000000,0.000000,0.000000,0.000000,0.000000
2,3,LiSrPO4,NaN,NaN,0.0,0.0,0.142857,0.0,0.0,0.0,...,0.285714,2.000000,0.000000,0.000000,0.714286,0.000000,0.000000,0.000000,0.000000,0.000000
3,4,LiSrSbO4,NaN,NaN,0.0,0.0,0.142857,0.0,0.0,0.0,...,0.285714,2.000000,0.000000,0.000000,0.714286,0.000000,0.000000,0.000000,0.000000,0.000000
4,5,LiCaAsO4,NaN,NaN,0.0,0.0,0.142857,0.0,0.0,0.0,...,0.285714,2.000000,0.000000,0.000000,0.714286,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,62,LiInGeO4,0.27989,S,0.0,0.0,0.142857,0.0,0.0,0.0,...,0.000000,1.285714,0.000000,0.000000,1.714286,0.000000,0.000000,0.000000,0.000000,0.000000
62,63,LiInSiO4,0.39616,S,0.0,0.0,0.142857,0.0,0.0,0.0,...,0.000000,1.285714,0.000000,0.000000,1.714286,0.000000,0.000000,0.000000,0.000000,0.000000
63,64,LiScGeO4,0.21838,S,0.0,0.0,0.142857,0.0,0.0,0.0,...,0.000000,1.000000,0.285714,0.000000,0.714286,1.000000,0.000000,0.000000,0.000000,0.000000
64,65,LiScSiO4,0.33218,S,0.0,0.0,0.142857,0.0,0.0,0.0,...,0.000000,1.000000,0.285714,0.000000,0.714286,1.000000,0.000000,0.000000,0.000000,0.000000
